In [ ]:
!pip -q install -U "gradio>=4.44,<5" "huggingface_hub<0.26.0"
import os, time; print("Restarting…"); time.sleep(1); os._exit(0)


Restarting…


In [1]:
!pip -q install "pycaret[full]>=3.0,<4" "gradio>=4.0,<5"

import pycaret, sys, sklearn
print("PyCaret:", pycaret.__version__)
print("sklearn:", sklearn.__version__)

PyCaret: 3.3.2
sklearn: 1.4.2


In [4]:
# Clean up the bad package and install the right one
%pip uninstall -y multipart
%pip install -U python-multipart gradio

# (Optional but recommended) restart the kernel so imports pick up newly installed deps
import IPython
IPython.get_ipython().kernel.do_shutdown(True)


Found existing installation: multipart 1.3.0
Uninstalling multipart-1.3.0:
  Successfully uninstalled multipart-1.3.0
Note: you may need to restart the kernel to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.6/68.6 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.5/63.5 MB 27.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.4/325.4 kB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.1/566.1 kB 34.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 444.8/444.8 kB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 71.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.3/182.3 kB 9.2 MB/s eta 0:00:00
  Attempting uninstall: websockets
    Found existing installation: websockets 12.0
    Uninstalling websockets-12.0:
      Successfully uninstalled websockets-12.0
  Attempting uninstall: pydantic-core
    Found existing installation: pydantic_c

{'status': 'ok', 'restart': True}

In [2]:
import pandas as pd
import gradio as gr
from pycaret.classification import load_model, predict_model


clf = load_model("/kaggle/input/pycaret-models-prachi/binclf_final")

feature_cols = [
    "age","job","marital","education","default","housing","loan",
    "contact","month","day_of_week","duration","campaign","pdays","previous",
    "poutcome","emp.var.rate","cons.price.idx","cons.conf.idx","euribor3m","nr.employed"
]

inputs = []
num_cols = {"age","duration","campaign","pdays","previous",
            "emp.var.rate","cons.price.idx","cons.conf.idx","euribor3m","nr.employed"}

for c in feature_cols:
    if c in num_cols:
        inputs.append(gr.Number(label=c))
    else:
        inputs.append(gr.Textbox(label=c))

def predict_row(*vals):
    row = pd.DataFrame([dict(zip(feature_cols, vals))])

    # ensure numeric fields are numeric (Gradio sometimes passes None/str)
    for c in num_cols:
        row[c] = pd.to_numeric(row[c], errors="coerce")

    out = predict_model(clf, data=row)

    # be tolerant to different column names across PyCaret versions
    label_col = next((c for c in ["prediction_label", "Label", "label"] if c in out.columns), None)
    score_col = next((c for c in ["prediction_score", "Score", "probability"] if c in out.columns), None)

    label = out[label_col].iloc[0] if label_col else "N/A"
    if score_col:
        try:
            score = float(out[score_col].iloc[0])
            return f"Prediction: {label}  |  Score: {score:.3f}"
        except Exception:
            return f"Prediction: {label}"
    else:
        return f"Prediction: {label}"

demo = gr.Interface(fn=predict_row, inputs=inputs, outputs=gr.Textbox(label="output"),
                    title="Bank Marketing — Binary Classifier")
demo.launch(share=True)


Transformation Pipeline and Model Successfully Loaded
* Running on local URL:  http://127.0.0.1:7861
* Running on public URL: https://a5778f9e4005451507.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=0.6, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=0.6, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6
